### 导入必要的库，简单设置

In [2]:
import pandas as pd

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.options.display.max_columns=99
sns.set(font='Microsoft YaHei Mono')
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

### 读取要处理的数据

In [9]:
??pd.read_csv

In [18]:
df = pd.read_csv('淄博信息港二手房详情20160110.csv', engine='python', encoding='utf8')


### 原始数据需要处理的问题
- 部分列存储了多列信息
    比如my_column_5列中的一个数据：户型：3室 2厅 1卫 楼层：18 /22，我们希望处理成：
   
| 室 | 厅 | 卫 | 所在层 | 总楼层 |
| :-: | :-: | :-: | :-: | :-: |
| 3 | 2 | 1 | 18 | 22 |

- 大部分列包含了不需要的字符：~~朝向：~~东、~~装修：~~毛坯……
- 处理完成之后需要做格式转换，发布时间应该是日期，价格、面积、室厅卫数量应该是数值……
- 需要计算出一些辅助信息，比如说根据建造年代生成房龄。

#### 由简至繁，先处理只需要剔除冗余字符的列。

In [23]:
new_df=pd.DataFrame(index=df.index)

In [24]:
new_df["区县"]= df['my_column'].str.replace("二手房","")  #新增一列
new_df["区县"].value_counts()

张店区    19437
临淄区      404
博山区      108
淄川区       85
周村区       64
桓台县       48
高青县       11
沂源县        2
           1
Name: 区县, dtype: int64

In [22]:
df.head(1)

,my_column_10,my_column_12,my_column_11,my_column_14,my_column,my_column_13,my_column_15,my_column_5,my_column_4,my_column_3,my_column_2,my_column_9,my_column_8,my_column_7,my_column_6,pageUrl
0,朝向：南北,周边学校：,装修：毛坯,2015-11-06发布,张店区二手房,13561686777,"室内设施：天然气,暖气,防盗门,阳台封闭 妇幼保健院新址不远，市总工会宿舍，惠泽苑小区，16...",户型：2室 2厅 1卫 楼层：4 /5,面积：120㎡ 车:15㎡,房主净得价 有营业税,￥51.00万元,建造年代：2010年,类型：住宅,地址：荷塘月色北邻，淄博市总工会后院,惠泽苑,http://house.zbinfo.net/sell_content.aspx?id=1...


*貌似只有张店人在用淄博信息港。*

In [25]:
new_df["小区"]= df['my_column_6']
new_df["小区"].value_counts().head(),new_df["小区"].value_counts().tail()

(世纪花园       1072
 黄金国际        905
 龙泰苑世嘉       537
 江南豪庭        536
 中润华侨城南区     520
 Name: 小区, dtype: int64, 安嘉公寓                1
 科技苑小区金花村（市财政局宿舍）    1
 市府东二街11号楼4单元501     1
 儿童公园附近单位房           1
 人民路57号院             1
 Name: 小区, dtype: int64)

In [28]:
new_df.head(3)

,区县,小区
0,张店区,惠泽苑
1,张店区,曹村创业园一号楼 曹三村 急需用钱超低价出售中介勿扰
2,张店区,南苑绿洲一区


*小区名称还需要整理，为了简化，可以舍弃名称不对的数据*

In [30]:
new_df["类型"]= df['my_column_8'].str.replace("类型：","")
new_df["类型"].value_counts()

住宅      19841
商铺        151
写字楼        82
别墅         49
车库         15
海景房        11
厂房仓库       11
Name: 类型, dtype: int64

In [31]:
new_df.head(3)

,区县,小区,类型
0,张店区,惠泽苑,住宅
1,张店区,曹村创业园一号楼 曹三村 急需用钱超低价出售中介勿扰,住宅
2,张店区,南苑绿洲一区,住宅


In [32]:
new_df["建造年代"]= df['my_column_9'].str.replace("建造年代：","").str.replace("年","")
new_df["建造年代"].value_counts()

2010    3810
2008    2080
2009    1553
2006    1400
2007    1310
2005    1245
2012    1177
2011     848
2004     827
2000     806
2013     666
2015     646
0        504
1998     482
2003     418
1999     357
2002     327
2014     298
1995     216
2001     183
1997     168
1996     168
1994     127
1993     120
1990      90
1992      48
1989      48
2016      39
1988      33
1987      33
1986      26
1991      26
1985      25
1984      15
1983      13
1982      11
1980      10
1981       2
1977       1
1512       1
1000       1
90         1
2017       1
Name: 建造年代, dtype: int64

In [33]:
new_df.head(3)

,区县,小区,类型,建造年代
0,张店区,惠泽苑,住宅,2010
1,张店区,曹村创业园一号楼 曹三村 急需用钱超低价出售中介勿扰,住宅,2006
2,张店区,南苑绿洲一区,住宅,2011


*2010年是盖房子高潮？*

In [34]:
df.head(1)

,my_column_10,my_column_12,my_column_11,my_column_14,my_column,my_column_13,my_column_15,my_column_5,my_column_4,my_column_3,my_column_2,my_column_9,my_column_8,my_column_7,my_column_6,pageUrl
0,朝向：南北,周边学校：,装修：毛坯,2015-11-06发布,张店区二手房,13561686777,"室内设施：天然气,暖气,防盗门,阳台封闭 妇幼保健院新址不远，市总工会宿舍，惠泽苑小区，16...",户型：2室 2厅 1卫 楼层：4 /5,面积：120㎡ 车:15㎡,房主净得价 有营业税,￥51.00万元,建造年代：2010年,类型：住宅,地址：荷塘月色北邻，淄博市总工会后院,惠泽苑,http://house.zbinfo.net/sell_content.aspx?id=1...


In [35]:
new_df["装修"]= df['my_column_11'].str.replace("装修：","")
new_df["装修"].value_counts()

精装修     6553
毛坯      5900
简装修     5608
中等装修    1179
豪华装修     893
原房        23
一般装修       2
毛坯房        1
           1
Name: 装修, dtype: int64

In [12]:
new_df["朝向"]= df['my_column_10'].str.replace("朝向：","")
new_df["朝向"].value_counts()

南     13246
南北     3390
东      3247
北       100
西        58
         58
东西       38
东南       13
西北        6
西南        3
东北        1
Name: 朝向, dtype: int64

*南北和南有什么区别？东西和东有什么区别？*

In [13]:
new_df["发布时间"]=pd.to_datetime(df['my_column_14'].str.replace("发布",""),format='%Y-%m-%d')
new_df["发布时间"].value_counts()

2016-01-07    811
2016-01-10    727
2016-01-09    620
2015-12-29    586
2016-01-08    583
2016-01-05    574
2016-01-03    547
2016-01-06    540
2016-01-02    493
2016-01-04    490
2015-12-28    431
2015-12-24    410
2015-12-27    382
2015-12-25    372
2015-12-26    371
2015-12-30    364
2016-01-01    268
2015-12-31    264
2015-12-13    252
2015-12-23    249
2015-12-22    223
2015-12-19    220
2015-12-20    212
2015-12-09    199
2015-12-12    193
2015-12-11    191
2015-12-15    190
2015-12-21    184
2015-12-14    183
2015-11-09    179
             ... 
2012-07-02      1
2014-10-09      1
2015-02-04      1
2015-05-16      1
2013-12-10      1
2013-10-04      1
2015-01-23      1
2013-05-28      1
2014-07-13      1
2013-12-19      1
2011-11-30      1
2014-11-09      1
2015-05-04      1
2013-05-01      1
2015-03-24      1
2013-08-22      1
2013-07-05      1
2014-04-26      1
2014-04-16      1
2015-02-26      1
2012-06-19      1
2015-05-03      1
2013-06-17      1
2014-02-21      1
2015-03-11

In [14]:
new_df["价格"]= pd.to_numeric(df['my_column_2'].str.replace("￥","").str.replace("万元",""),errors='coerce')
new_df["价格"].value_counts() 

85.00     503
75.00     485
95.00     391
65.00     380
55.00     372
110.00    343
58.00     341
76.00     334
105.00    330
90.00     327
63.00     315
88.00     311
60.00     306
82.00     297
70.00     292
78.00     291
115.00    278
100.00    272
66.00     257
80.00     256
92.00     256
50.00     255
72.00     242
53.00     237
93.00     233
68.00     231
56.00     226
108.00    219
73.00     217
62.00     208
         ... 
61.60       1
18.70       1
6.60        1
44.30       1
103.40      1
138.80      1
228.00      1
95.90       1
240.24      1
167.00      1
251.00      1
164.00      1
135.30      1
312.00      1
161.00      1
329.00      1
29.99       1
6.80        1
1.80        1
9.80        1
10.80       1
153.00      1
15.80       1
21.80       1
30.30       1
31.80       1
86.90       1
89.90       1
91.90       1
0.50        1
Name: 价格, dtype: int64

In [15]:
len(new_df[new_df['价格'].isnull()])

58

#### 接下来处理稍微复杂些的，需要拆分的列

**面积** 
- 面积：120㎡ 车:15㎡
- 面积：130㎡ 储:17㎡
- ……

In [36]:
df.head(1)

,my_column_10,my_column_12,my_column_11,my_column_14,my_column,my_column_13,my_column_15,my_column_5,my_column_4,my_column_3,my_column_2,my_column_9,my_column_8,my_column_7,my_column_6,pageUrl
0,朝向：南北,周边学校：,装修：毛坯,2015-11-06发布,张店区二手房,13561686777,"室内设施：天然气,暖气,防盗门,阳台封闭 妇幼保健院新址不远，市总工会宿舍，惠泽苑小区，16...",户型：2室 2厅 1卫 楼层：4 /5,面积：120㎡ 车:15㎡,房主净得价 有营业税,￥51.00万元,建造年代：2010年,类型：住宅,地址：荷塘月色北邻，淄博市总工会后院,惠泽苑,http://house.zbinfo.net/sell_content.aspx?id=1...


In [39]:
s1="面积：120㎡ 车:15㎡"
s2="面积：130㎡ 储:17㎡"

def deal_str(s,k):
    d={"面积":np.nan,"车":np.nan,"储":np.nan}
    if type(s)==type(""):# pandas 中的Nan值是flaot类型的，如果出现缺失值，当成字符处理就会出错。
        s=s.replace("㎡","").replace("：",":")
        l=s.split(" ")
        #print('l-->', l)
        for i in l:
            tmpl=i.split(":")
            #print('tmp->', tmpl)
            d[tmpl[0]]=tmpl[1]
    return d[k]

def get_area(s):
    return deal_str(s,k="面积")
def get_car(s):
    return deal_str(s,k="车")
def get_store(s):
    return deal_str(s,k="储")
    
        
get_area(s1),get_area(s2)

('120', '130')

In [40]:
new_df["面积"]=df["my_column_4"].map(get_area)
new_df["车库"]=df["my_column_4"].map(get_car)
new_df["储藏室"]=df["my_column_4"].map(get_store)
new_df.head()

,区县,小区,类型,建造年代,装修,面积,车库,储藏室
0,张店区,惠泽苑,住宅,2010,毛坯,120,15,NaN
1,张店区,曹村创业园一号楼 曹三村 急需用钱超低价出售中介勿扰,住宅,2006,简装修,73,NaN,6
2,张店区,南苑绿洲一区,住宅,2011,简装修,82,NaN,10
3,张店区,阳光馨园,住宅,2009,中等装修,75,25,NaN
4,张店区,张店齐悦国际，改合同（划片祥瑞苑小学、张店八中）,住宅,2015,毛坯,140,18,20


**户型和楼层** 
- 户型：2室 2厅 1卫 楼层：4 /5 
- ……

In [42]:
import re

In [44]:
s="户型：2室 2厅 1卫 楼层：4 /5Fefefeefe"
r=r".*户型：(\d+)室\s+(\d+)厅\s+(\d+)卫\s+楼层：(\d+)\s\/(\d+)"
m=re.match(r,s)
print(m.group(0))
m.group(1),m.group(2),m.group(3),m.group(4),m.group(5)

户型：2室 2厅 1卫 楼层：4 /5


('2', '2', '1', '4', '5')

In [46]:
new_df["室"]=df['my_column_5'].map(lambda x: re.match(r".*(\d+)室",str(x)).group(1) if re.match(r".*户型：(\d+)室",str(x)) else np.nan)
new_df["室"].value_counts()

3    12196
2     5691
4     1248
1      566
5      340
6       80
0       15
8       13
7        7
9        4
Name: 室, dtype: int64

In [47]:
new_df["厅"]=df['my_column_5'].map(lambda x: re.match(r".*(\d+)厅",str(x)).group(1) if re.match(r".*(\d+)厅",str(x)) else np.nan)
new_df["厅"].value_counts()

2    17160
1     2293
3      538
0       88
4       61
5       11
6        7
7        2
Name: 厅, dtype: int64

In [48]:
new_df["卫"]=df['my_column_5'].map(lambda x: re.match(r".*(\d+)卫",str(x)).group(1) if re.match(r".*(\d+)卫",str(x)) else np.nan)
new_df["卫"].value_counts()

1    15815
2     3833
3      262
0      210
4       28
7        5
5        4
6        3
Name: 卫, dtype: int64

In [49]:
new_df["楼层"]=df['my_column_5'].map(lambda x: re.match(r".*楼层：(\d+)",str(x)).group(1) if re.match(r".*楼层：(\d+)",str(x)) else np.nan)
new_df["楼层"].value_counts()

3      2492
5      2483
6      2477
1      2297
2      2292
4      2283
7      1088
9       825
10      755
8       551
11      545
14      315
16      289
15      284
12      253
13      225
17      210
18       92
20       67
19       64
21       62
25       52
22       34
24       31
23       27
26       18
28        9
29        9
27        7
32        6
30        3
33        2
0         2
35        1
31        1
340       1
301       1
34        1
Name: 楼层, dtype: int64

In [50]:
new_df["总楼层"]=df['my_column_5'].map(lambda x: re.match(r".*楼层：\d+\s+/(\d+)",str(x)).group(1) if re.match(r".*楼层：\d+\s+/(\d+)",str(x)) else np.nan)
new_df["总楼层"].value_counts()

6      7565
7      3360
11     2618
17     1577
18     1028
5       573
12      469
16      384
15      303
21      251
4       249
23      164
24      158
3       132
22      129
19      124
9       114
1       109
33      103
13      101
2        90
20       75
30       66
10       63
28       53
26       50
32       46
14       41
25       40
34       26
29       23
27       18
8        16
31       16
35        9
63        3
0         2
199       1
120       1
41        1
73        1
40        1
118       1
Name: 总楼层, dtype: int64

In [51]:
new_df.head()

,区县,小区,类型,建造年代,装修,面积,车库,储藏室,室,厅,卫,楼层,总楼层
0,张店区,惠泽苑,住宅,2010,毛坯,120,15,NaN,2,2,1,4,5
1,张店区,曹村创业园一号楼 曹三村 急需用钱超低价出售中介勿扰,住宅,2006,简装修,73,NaN,6,2,1,1,4,6
2,张店区,南苑绿洲一区,住宅,2011,简装修,82,NaN,10,2,1,1,2,6
3,张店区,阳光馨园,住宅,2009,中等装修,75,25,NaN,3,1,1,1,6
4,张店区,张店齐悦国际，改合同（划片祥瑞苑小学、张店八中）,住宅,2015,毛坯,140,18,20,3,2,1,18,22


In [27]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20161 entries, 0 to 20160
Data columns (total 16 columns):
区县      20160 non-null object
小区      17869 non-null object
类型      20160 non-null object
建造年代    20160 non-null object
装修      20160 non-null object
朝向      20160 non-null object
发布时间    20160 non-null datetime64[ns]
价格      20103 non-null float64
面积      20160 non-null object
车库      830 non-null object
储藏室     2520 non-null object
室       20160 non-null object
厅       20160 non-null object
卫       20160 non-null object
楼层      20154 non-null object
总楼层     20154 non-null object
dtypes: datetime64[ns](1), float64(1), object(14)
memory usage: 2.6+ MB


In [28]:
new_df[["建造年代","面积","车库","储藏室","室","厅","卫","楼层","总楼层"]]=new_df[["建造年代","面积","车库","储藏室","室","厅","卫","楼层","总楼层"]].astype(float)

In [29]:
new_df[["车库","储藏室","室","厅","卫"]]=new_df[["车库","储藏室","室","厅","卫"]].fillna(0)

In [30]:
new_df.to_excel("淄博信息港二手房信息20160110my.xlsx")